# Workfile Examples

Let's run a workflow with workflow variable input and download the results locally.

In [1]:
username = 'demoadmin'
password = 'password'
host = "10.10.0.204"
port = "8080"

test_workflow_name = "WineData"
test_workspace_name = "APITests"

In [2]:
import api as AlpineAPI
from pprint import pprint

In [3]:
session = AlpineAPI.Alpine(host, port, username, password)

## Finding our workflow

In [4]:
test_workspace_id = session.workspace.get_id(test_workspace_name)
all_workfiles = session.workfile.get_list(test_workspace_id)
pprint(all_workfiles, depth = 2)

[{u'associated_worklets': [],
  u'comment_count': 0,
  u'complete_json': True,
  u'description': u'',
  u'entity_subtype': u'alpine',
  u'entity_type': u'workfile',
  u'file_name': u'Test workflow',
  u'file_type': u'work_flow',
  u'id': 7,
  u'is_deleted': None,
  u'latest_version_id': None,
  u'link_url': None,
  u'owner': {...},
  u'recent_comments': [],
  u'status': u'idle',
  u'tags': [],
  u'user_modified_at': u'2016-12-20T00:41:27Z',
  u'version_info': {...},
  u'workspace': {...}},
 {u'associated_worklets': [],
  u'comment_count': 0,
  u'complete_json': True,
  u'description': u'',
  u'entity_subtype': u'alpine',
  u'entity_type': u'workfile',
  u'file_name': u'WineData',
  u'file_type': u'work_flow',
  u'id': 375,
  u'is_deleted': None,
  u'latest_version_id': None,
  u'link_url': None,
  u'owner': {...},
  u'recent_comments': [],
  u'status': u'idle',
  u'tags': [],
  u'user_modified_at': u'2017-02-07T22:28:29Z',
  u'version_info': {...},
  u'workspace': {...}}]


In [5]:
test_workflow_id = session.workfile.get_id(test_workflow_name, test_workspace_id)
single_workfile_info = session.workfile.get(test_workflow_id)
pprint(single_workfile_info, depth = 2)

{u'response': {u'associated_worklets': [],
               u'comment_count': 0,
               u'complete_json': True,
               u'dataset_ids': [],
               u'description': u'',
               u'entity_subtype': u'alpine',
               u'entity_type': u'workfile',
               u'execution_locations': [...],
               u'file_name': u'WineData',
               u'file_type': u'work_flow',
               u'id': 375,
               u'is_deleted': None,
               u'latest_version_id': None,
               u'link_url': None,
               u'owner': {...},
               u'recent_comments': [],
               u'status': u'idle',
               u'tags': [],
               u'user_modified_at': u'2017-02-07T22:28:29Z',
               u'version_info': {...},
               u'workspace': {...}}}


## Running a workflow

In [6]:
process_id = session.workfile.process.run(test_workflow_id)

In [7]:
session.workfile.process.query_status(process_id)

'WORKING'

In [8]:
session.workfile.process.wait_until_finished(process_id, verbose=True, query_time=5, timeout=100)

Workflow in progress for ~0.08 seconds.

Workflow in progress for ~5.12 seconds.

Workflow in progress for ~10.17 seconds.

Workflow in progress for ~15.23 seconds.

Workflow in progress for ~20.32 seconds.

Workflow in progress for ~25.67 seconds.

Workflow in progress for ~30.72 seconds.

Workflow in progress for ~35.77 seconds.

'FINISHED'

## Downloading the flow results

In [9]:
flow_results = session.workfile.process.download_results(test_workflow_id, process_id)

In [10]:
operator_data = session.workfile.process.find_operator('Row Filter', flow_results);
pprint(operator_data, depth = 2)

{u'isGenerateReport': True,
 u'node_meta_info': [[...], [...], [...], [...], [...]],
 u'out_id': 1487725651921.7632,
 u'out_title': u'Row Filter',
 u'visualData': {u'columnTypes': [...],
                 u'columns': [...],
                 u'errorMessages': [...],
                 u'fixedNumberOfColumns': 0,
                 u'items': [...]},
 u'visualType': 0}


In [11]:
pprint(session.workfile.process.get_flow_metadata(flow_results))

{u'endTime': u'2017-02-21T17:07:49.353-0800',
 u'executeUser': u'665',
 u'noOfError': 0,
 u'noOfNodesProcessed': 3,
 u'processId': u'70c85fcd-e142-4111-94dd-f596ad74f366',
 u'startTime': u'2017-02-21T17:07:09.268-0800',
 u'status': u'SUCCESS',
 u'workflowId': u'375',
 u'workflowName': u'WineData'}


## Using workflow variables

After properly defining a workflow variable in Alpine UI, a user can inject different values into a workflow. In this workflow I've defined a workflow variable that is being used in a Row Filter operator.

In [13]:
variables = [{"name": "@row_filter", "value": "13"}]
process_id = session.workfile.process.run(test_workflow_id, variables=variables)
session.workfile.process.wait_until_finished(process_id, verbose=True, query_time=5, timeout=100)

Workflow in progress for ~0.04 seconds.

Workflow in progress for ~5.08 seconds.

Workflow in progress for ~10.14 seconds.

Workflow in progress for ~15.19 seconds.

Workflow in progress for ~20.23 seconds.

Workflow in progress for ~25.36 seconds.

Workflow in progress for ~30.44 seconds.

Workflow in progress for ~35.49 seconds.

Workflow in progress for ~40.63 seconds.

'FINISHED'

In [14]:
flow_results = session.workfile.process.download_results(test_workflow_id, process_id)
ss_results = session.workfile.process.find_operator("Summary Statistics", flow_results)
pprint(ss_results['visualData']['items'][0]['count'])

u'86'


In [15]:
ss_results = session.workfile.process.download_results(test_workflow_id, process_id, "Summary Statistics")
pprint(ss_results['visualData']['items'][0]['count'])

u'86'
